In [15]:
import asf_search as asf
import numpy as np
import geopandas as gpd
import pandas as pd

from bokeh.sampledata.sample_geojson import geojson
import datetime
from bokeh.io import curdoc, output_notebook
from bokeh.layouts import column, row
from bokeh.models import GeoJSONDataSource, DataRange1d, Select
from bokeh.palettes import Blues4
from bokeh.plotting import figure,show
import json

output_notebook()

Loading BokehJS ...

In [16]:
def make_df(results):
    df = pd.DataFrame(
    {
        'fileID':[],
        'platform':[],
        'flightDirection':[],
        'centerLat':[],
        'centerLon':[],
#         'processingLevel':[],
        'startTime':[]
    })
    name = []
    platformlist = []
    flightlist = []
    latlist = []
    lonlist = []
#     processlist = []
    stime = []
    for i in range(len(results)):
        name.append(results[i].properties['fileID'])
        platformlist.append(results[i].properties['platform'])
        flightlist.append(results[i].properties['flightDirection'])
        latlist.append(float(results[i].properties['centerLat']))
        lonlist.append(float(results[i].properties['centerLon']))
#         processlist.append(results[i].properties['processingLevel'])
        timestring = results[i].properties['startTime'][0:10] + ' ' + results[i].properties['startTime'][11:]
        stime.append(timestring)

    df.fileID = name
    df.platform = platformlist
    df.flightDirection = flightlist
    df.centerLat = latlist
    df.centerLon = lonlist
#     df.processingLevel = processlist
    df.startTime = stime
    
    return df

In [17]:
opts = {
    'platform': asf.PLATFORM.SENTINEL1,
    'maxResults': 100,
    'start': '2016-01-01T00:00:00Z',
    'end': '2016-12-31T23:59:59Z'
}
results = asf.search(**opts)

df_2016 = make_df(results)
gdf = gpd.GeoDataFrame(df_2016, geometry=gpd.points_from_xy(df_2016.centerLon, df_2016.centerLat))

In [18]:
type(results)

asf_search.ASFSearchResults.ASFSearchResults

In [19]:
gdf.drop(['geometry'], axis=1,inplace=True)

In [20]:
gdf.to_csv('Sentinel1_2016.csv')

In [21]:
def get_dataset(src):
    gdf = src.copy()
    gdf['date'] = gdf['startTime']
    gdf = gdf.set_index(['date'])
    gdf.sort_index(inplace=True)
    gdf = gdf.to_dict()

    return GeoJSONDataSource(geojson=json.dumps(gdf))

In [22]:
def make_plot(source, title):
    #     plot = figure(x_axis_type="datetime", width=800, tools="", toolbar_location=None)
    plot = figure()
    plot.title.text = title

    plot.line('date','centerLat',source=source)

    # fixed attributes
    #     plot.xaxis.axis_label = None
    #     plot.yaxis.axis_label = "Latitude"
    #     plot.axis.axis_label_text_font_style = "bold"
    #     plot.x_range = DataRange1d(range_padding=0.0)
    #     plot.grid.grid_line_alpha = 0.3

    return plot

In [23]:
def update_plot(attrname, old, new):
    city = city_select.value
    plot.title.text = "Weather data for " + cities[city]['title']

    src = get_dataset(df, cities[city]['airport'], distribution_select.value)
    source.data.update(src.data)

In [24]:
source = get_dataset(gdf)

In [25]:
plot = make_plot(source, "Ground-swath project")

curdoc().add_root(row(plot))
curdoc().title = "Ground-swath project"

inline

In [52]:
def lnglat_to_meters(longitude, latitude):
    """
    Projects the given (longitude, latitude) values into Web Mercator
    coordinates (meters East of Greenwich and meters North of the Equator).

    Longitude and latitude can be provided as scalars, Pandas columns,
    or Numpy arrays, and will be returned in the same form.  Lists
    or tuples will be converted to Numpy arrays.

    Examples:
       easting, northing = lnglat_to_meters(-74,40.71)

       easting, northing = lnglat_to_meters(np.array([-74]),np.array([40.71]))

       df=pandas.DataFrame(dict(longitude=np.array([-74]),latitude=np.array([40.71])))
       df.loc[:, 'longitude'], df.loc[:, 'latitude'] = lnglat_to_meters(df.longitude,df.latitude)
    """
    if isinstance(longitude, (list, tuple)):
        longitude = np.array(longitude)
    if isinstance(latitude, (list, tuple)):
        latitude = np.array(latitude)

    origin_shift = np.pi * 6378137
    easting = longitude * origin_shift / 180.0
    northing = np.log(np.tan((90 + latitude) * np.pi / 360.0)) * origin_shift / np.pi
    return (easting, northing)

In [56]:
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '463098',
   'geometry': {'type': 'Point',
    'coordinates': [-2.1208465099334717, 51.4613151550293]},
   'properties': {'OrganisationCode': 'Q64',
    'OrganisationType': 'Area Team',
    'SubType': 'UNKNOWN',
    'OrganisationStatus': 'Visible',
    'IsPimsManaged': 'True',
    'OrganisationName': 'Bath, Gloucestershire, Swindon And Wiltshire Area Team',
    'Address1': '1st Floor',
    'Address2': 'Bewley House',
    'Address3': 'Marshfield Road',
    'City': 'Chippenham',
    'County': 'Wiltshire',
    'Postcode': 'SN15 1JW',
    'Phone': '0113 8251 500',
    'Email': 'england.contactus@nhs.net',
    'Website': 'http://www.england.nhs.uk/south/south/bgsw-at/',
    'Color': 'blue'}},
  {'type': 'Feature',
   'id': '463099',
   'geometry': {'type': 'Point',
    'coordinates': [-2.5929524898529053, 51.459877014160156]},
   'properties': {'OrganisationCode': 'Q65',
    'OrganisationType': 'Area Team',
    'SubTyp

In [57]:
from bokeh.plotting import figure, output_file, show
from bokeh.tile_providers import Vendors, get_provider

tile_provider = get_provider(Vendors.CARTODBPOSITRON)

TOOLTIPS = [
    ('Organisation', '@OrganisationName')
]
# range bounds supplied in web mercator coordinates
p = figure(x_range=(-2000000, 6000000), y_range=(-1000000, 7000000),
           x_axis_type="mercator", y_axis_type="mercator", tooltips=TOOLTIPS)
p.add_tile(tile_provider)

data = json.loads(geojson)
for i in range(len(data['features'])):
    data['features'][i]['properties']['Color'] = ['blue', 'red'][i%2]
    lon, lat = data['features'][i]['geometry']['coordinates']
    data['features'][i]['geometry']['coordinates'] = lnglat_to_meters(lon, lat)

geo_source = GeoJSONDataSource(geojson=json.dumps(data))


# p = figure(background_fill_color="lightgrey", tooltips=TOOLTIPS)
p.circle(x='x', y='y', size=15, color='Color', alpha=0.7, source=geo_source)

show(p)

In [42]:
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '463098',
   'geometry': {'type': 'Point',
    'coordinates': [-2.1208465099334717, 51.4613151550293]},
   'properties': {'OrganisationCode': 'Q64',
    'OrganisationType': 'Area Team',
    'SubType': 'UNKNOWN',
    'OrganisationStatus': 'Visible',
    'IsPimsManaged': 'True',
    'OrganisationName': 'Bath, Gloucestershire, Swindon And Wiltshire Area Team',
    'Address1': '1st Floor',
    'Address2': 'Bewley House',
    'Address3': 'Marshfield Road',
    'City': 'Chippenham',
    'County': 'Wiltshire',
    'Postcode': 'SN15 1JW',
    'Phone': '0113 8251 500',
    'Email': 'england.contactus@nhs.net',
    'Website': 'http://www.england.nhs.uk/south/south/bgsw-at/',
    'Color': 'blue'}},
  {'type': 'Feature',
   'id': '463099',
   'geometry': {'type': 'Point',
    'coordinates': [-2.5929524898529053, 51.459877014160156]},
   'properties': {'OrganisationCode': 'Q65',
    'OrganisationType': 'Area Team',
    'SubTyp

In [27]:
data['features']

[{'type': 'Feature',
  'id': '463098',
  'geometry': {'type': 'Point',
   'coordinates': [-2.1208465099334717, 51.4613151550293]},
  'properties': {'OrganisationCode': 'Q64',
   'OrganisationType': 'Area Team',
   'SubType': 'UNKNOWN',
   'OrganisationStatus': 'Visible',
   'IsPimsManaged': 'True',
   'OrganisationName': 'Bath, Gloucestershire, Swindon And Wiltshire Area Team',
   'Address1': '1st Floor',
   'Address2': 'Bewley House',
   'Address3': 'Marshfield Road',
   'City': 'Chippenham',
   'County': 'Wiltshire',
   'Postcode': 'SN15 1JW',
   'Phone': '0113 8251 500',
   'Email': 'england.contactus@nhs.net',
   'Website': 'http://www.england.nhs.uk/south/south/bgsw-at/',
   'Color': 'blue'}},
 {'type': 'Feature',
  'id': '463099',
  'geometry': {'type': 'Point',
   'coordinates': [-2.5929524898529053, 51.459877014160156]},
  'properties': {'OrganisationCode': 'Q65',
   'OrganisationType': 'Area Team',
   'SubType': 'UNKNOWN',
   'OrganisationStatus': 'Visible',
   'IsPimsManaged'

In [28]:
geo_source.geojson

'{"type": "FeatureCollection", "features": [{"type": "Feature", "id": "463098", "geometry": {"type": "Point", "coordinates": [-2.1208465099334717, 51.4613151550293]}, "properties": {"OrganisationCode": "Q64", "OrganisationType": "Area Team", "SubType": "UNKNOWN", "OrganisationStatus": "Visible", "IsPimsManaged": "True", "OrganisationName": "Bath, Gloucestershire, Swindon And Wiltshire Area Team", "Address1": "1st Floor", "Address2": "Bewley House", "Address3": "Marshfield Road", "City": "Chippenham", "County": "Wiltshire", "Postcode": "SN15 1JW", "Phone": "0113 8251 500", "Email": "england.contactus@nhs.net", "Website": "http://www.england.nhs.uk/south/south/bgsw-at/", "Color": "blue"}}, {"type": "Feature", "id": "463099", "geometry": {"type": "Point", "coordinates": [-2.5929524898529053, 51.459877014160156]}, "properties": {"OrganisationCode": "Q65", "OrganisationType": "Area Team", "SubType": "UNKNOWN", "OrganisationStatus": "Visible", "IsPimsManaged": "True", "OrganisationName": "Br